In [14]:
from copy import deepcopy
from time import time

import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd

from torchvision.datasets import FashionMNIST
from torchvision import transforms

from ray import tune

In [15]:
tune.run?

In [3]:
def acc(net_output, labels):
    predicted = net_output.argmax(dim=1)
    correct = (predicted == labels).sum()
    examples = len(labels)
    return (correct / examples).item()


class FMnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [4]:
ds_train = FashionMNIST(
    root="./data", 
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0, 1)
    ]), 
    download=True, 
    train=True)

ds_test = FashionMNIST(
    root="./data", 
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0, 1)
    ]), 
    download=True,
    train=False)

/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
train_set = DataLoader(ds_train, batch_size=64, shuffle=True, num_workers=4)
test_set = DataLoader(ds_test, batch_size=len(ds_train))

In [6]:
net = FMnistNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

total_epochs = 0
train_loss_progress = []
test_loss_progress = []
train_acc_progress = []
test_acc_progress = []

In [8]:
epochs_per_run=2
total_epochs += epochs_per_run

for epoch in range(total_epochs-epochs_per_run, total_epochs):
    epoch_start = time()
    
    accum_loss = []
    accum_acc = []
    
    for data, labels in train_set:
        optimizer.zero_grad()
        net_output = net(data)
        loss = loss_fn(net_output, labels)
        loss.backward()
        optimizer.step()
        
        accum_loss.append(loss.item())
        accum_acc.append(acc(net_output, labels))
    
    epoch_duration = time() - epoch_start
        
    train_avg_loss = np.mean(accum_loss)
    train_avg_acc = np.mean(accum_acc)
    
    with torch.no_grad():
        test_data, test_labels = next(iter(test_set))
        test_output = net(test_data)
        test_loss = loss_fn(test_output, test_labels)
        test_acc = acc(test_output, test_labels)    
        print(test_acc, type(test_acc))
        
    train_loss_progress.append(train_avg_loss)
    test_loss_progress.append(test_loss.item())
    train_acc_progress.append(train_avg_acc)
    test_acc_progress.append(test_acc)
    
#     print(f"EPOCH {epoch+1}/{total_epochs} -- train: (loss: {train_avg_loss:.4f}, acc: {100*train_avg_acc:.2f}%), test: (loss: {test_loss:.4f}, acc: {100*test_acc:.2f}%), time: {epoch_duration:.2f}s")

0.7922999858856201 <class 'float'>


KeyboardInterrupt: 

In [ ]:
metadata_df = pd.DataFrame((train_loss_progress, test_loss_progress, train_acc_progress, test_acc_progress)).T
metadata_df.columns = ["train_loss", "test_loss", "train_acc", "test_acc"]
metadata_df['epoch'] = np.arange(1, total_epochs+1)
metadata_df = metadata_df.set_index('epoch')

In [ ]:
metadata_df[['train_loss', 'test_loss']].plot(kind='line')

In [ ]:
metadata_df[['train_acc', 'test_acc']].plot()